In [27]:
import numpy  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot
from subprocess import check_output
from keras.layers.core import Dense, Activation, Dropout
from sklearn.metrics import mean_squared_error
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from sklearn.cross_validation import  train_test_split
import time #helper libraries
from sklearn.preprocessing import MinMaxScaler
from numpy import newaxis

In [2]:
series = pd.read_csv('output3rdformat.csv')
X = series.values
print(series.columns)
print(series)
print(series.describe())
series = series.astype('float32')
series

Index(['Month', 'Fuel Oil Consumption(Predicted)',
       'Cooling Degree Days U.S. Average', 'Heating Degree Days U.S. Average ',
       'Fuel Oil Consumption(Actual)'],
      dtype='object')
     Month  Fuel Oil Consumption(Predicted)  Cooling Degree Days U.S. Average  \
0        1                         0.894624                          0.003067   
1        2                         1.049869                          0.009975   
2        3                         0.640267                          0.012469   
3        4                         0.428591                          0.064838   
4        5                         0.396877                          0.064838   
5        6                         0.297866                          0.329177   
6        7                         0.400244                          0.496259   
7        8                         0.373951                          0.725686   
8        9                         0.449943                          0.610973 

,Month,Fuel Oil Consumption(Predicted),Cooling Degree Days U.S. Average,Heating Degree Days U.S. Average,Fuel Oil Consumption(Actual)
0,1.0,0.894624,0.003067,0.795156,0.97
1,2.0,1.049869,0.009975,2.406484,0.84
2,3.0,0.640267,0.012469,1.902743,0.63
3,4.0,0.428591,0.064838,1.234414,0.49
4,5.0,0.396877,0.064838,0.770574,0.35
5,6.0,0.297866,0.329177,0.284289,0.34
6,7.0,0.400244,0.496259,0.094763,0.32
7,8.0,0.373951,0.725686,0.014963,0.36
8,9.0,0.449943,0.610973,0.047382,0.39
9,10.0,0.459995,0.418953,0.124688,0.46


In [3]:
y1= series['Fuel Oil Consumption(Actual)']
X1= series.drop('Fuel Oil Consumption(Actual)',axis = 1)

In [4]:
y1=numpy.array(y1)
X1=numpy.array(X1)

In [5]:
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
y1 = scaler.fit_transform(y1.reshape(-1,1))
print(y1)
print(len(y1))
X1=scaler.fit_transform(X1)
print(X1)

[[1.        ]
 [0.85393256]
 [0.6179775 ]
 [0.46067417]
 [0.30337077]
 [0.29213482]
 [0.26966292]
 [0.31460673]
 [0.34831458]
 [0.4269663 ]
 [0.5280899 ]
 [0.7303371 ]
 [0.79775286]
 [0.752809  ]
 [0.6404494 ]
 [0.38202244]
 [0.33707863]
 [0.30337077]
 [0.24719103]
 [0.32584268]
 [0.33707863]
 [0.3707865 ]
 [0.516854  ]
 [0.7303371 ]
 [0.6966292 ]
 [0.752809  ]
 [0.6404494 ]
 [0.34831458]
 [0.26966292]
 [0.22471909]
 [0.20224719]
 [0.24719103]
 [0.24719103]
 [0.32584268]
 [0.40449435]
 [0.46067417]
 [0.74157304]
 [0.94382024]
 [0.6516854 ]
 [0.43820226]
 [0.26966292]
 [0.19101124]
 [0.15730336]
 [0.17977528]
 [0.16853933]
 [0.23595504]
 [0.4269663 ]
 [0.6179775 ]
 [0.752809  ]
 [0.7303371 ]
 [0.5280899 ]
 [0.30337077]
 [0.16853933]
 [0.13483146]
 [0.12359551]
 [0.14606741]
 [0.19101124]
 [0.31460673]
 [0.41573036]
 [0.6966292 ]
 [0.8651686 ]
 [0.6966292 ]
 [0.516854  ]
 [0.28089887]
 [0.13483146]
 [0.1011236 ]
 [0.0786517 ]
 [0.06741573]
 [0.14606741]
 [0.25842696]
 [0.29213482]
 [0.59

In [31]:
n_train=4
i=0
for j in range(2):
    
    X_train1 = X1[i:4,:]
    X_test1 = X1[3+i:4+i,:]
    print(X_train1.shape)
    print(X_test1.shape)
    y_train1 = y1[:4+i]
    y_test1 = y1[3+i:4+i]
    print(y_train1.shape)
    print(y_test1.shape)
    X_train=numpy.array(X_train1)
    y_train=numpy.array(y_train1)
    X_test=numpy.array(X_test1)
    y_test=numpy.array(y_test1)
    X_train = numpy.reshape(X_train1, (X_train1.shape[0], X_train1.shape[1], 1))
    X_test = numpy.reshape(X_test1, (X_test1.shape[0], X_test1.shape[1], 1))
    #Step 2 Build Model
    model = Sequential()

    model.add(LSTM(50,input_shape=(4, 1),return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(100,return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(units=1))
    model.add(Activation('linear'))

    start = time.time()
    model.compile(loss='mse', optimizer='adam', metrics = ['accuracy'])
    print ('compilation time : ', time.time() - start)
    model.fit(X_train,y_train,batch_size=1,epochs=300,verbose=2)
    #make predictions with the model
    trainPredict = model.predict(X_train)
    testPredict = model.predict(X_test)
    print(trainPredict)
    print(testPredict)
    trainScore = mean_squared_error(y_train,trainPredict)
    print('Train Score: %.4f MSE' % (trainScore))
    testScore = mean_squared_error(y_test,testPredict)
    print('Test Score: %.4f MSE' % (testScore))
   
i=i+5

(4, 4)
(1, 4)
(4, 1)
(1, 1)
compilation time :  0.029579639434814453
Epoch 1/300
 - 4s - loss: 0.5433 - acc: 0.0000e+00
Epoch 2/300
 - 0s - loss: 0.4343 - acc: 0.0000e+00
Epoch 3/300
 - 0s - loss: 0.3188 - acc: 0.0000e+00
Epoch 4/300
 - 0s - loss: 0.2236 - acc: 0.0000e+00
Epoch 5/300
 - 0s - loss: 0.1079 - acc: 0.0000e+00
Epoch 6/300
 - 0s - loss: 0.0191 - acc: 0.2500
Epoch 7/300
 - 0s - loss: 0.0486 - acc: 0.2500
Epoch 8/300
 - 0s - loss: 0.0792 - acc: 0.2500
Epoch 9/300
 - 0s - loss: 0.0729 - acc: 0.2500
Epoch 10/300
 - 0s - loss: 0.0480 - acc: 0.2500
Epoch 11/300
 - 0s - loss: 0.0396 - acc: 0.2500
Epoch 12/300
 - 0s - loss: 0.0549 - acc: 0.2500
Epoch 13/300
 - 0s - loss: 0.0514 - acc: 0.2500
Epoch 14/300
 - 0s - loss: 0.0349 - acc: 0.2500
Epoch 15/300
 - 0s - loss: 0.0348 - acc: 0.2500
Epoch 16/300
 - 0s - loss: 0.0530 - acc: 0.2500
Epoch 17/300
 - 0s - loss: 0.0375 - acc: 0.2500
Epoch 18/300
 - 0s - loss: 0.0327 - acc: 0.2500
Epoch 19/300
 - 0s - loss: 0.0542 - acc: 0.2500
Epoch 20

Epoch 169/300
 - 0s - loss: 0.0357 - acc: 0.2500
Epoch 170/300
 - 0s - loss: 0.0445 - acc: 0.2500
Epoch 171/300
 - 0s - loss: 0.0491 - acc: 0.2500
Epoch 172/300
 - 0s - loss: 0.0443 - acc: 0.2500
Epoch 173/300
 - 0s - loss: 0.0387 - acc: 0.2500
Epoch 174/300
 - 0s - loss: 0.0302 - acc: 0.2500
Epoch 175/300
 - 0s - loss: 0.0297 - acc: 0.2500
Epoch 176/300
 - 0s - loss: 0.0449 - acc: 0.2500
Epoch 177/300
 - 0s - loss: 0.0380 - acc: 0.2500
Epoch 178/300
 - 0s - loss: 0.0657 - acc: 0.2500
Epoch 179/300
 - 0s - loss: 0.0484 - acc: 0.2500
Epoch 180/300
 - 0s - loss: 0.0419 - acc: 0.2500
Epoch 181/300
 - 0s - loss: 0.0480 - acc: 0.2500
Epoch 182/300
 - 0s - loss: 0.0367 - acc: 0.2500
Epoch 183/300
 - 0s - loss: 0.0206 - acc: 0.2500
Epoch 184/300
 - 0s - loss: 0.0327 - acc: 0.2500
Epoch 185/300
 - 0s - loss: 0.0282 - acc: 0.2500
Epoch 186/300
 - 0s - loss: 0.0382 - acc: 0.2500
Epoch 187/300
 - 0s - loss: 0.0423 - acc: 0.2500
Epoch 188/300
 - 0s - loss: 0.0349 - acc: 0.2500
Epoch 189/300
 - 0s 

Epoch 33/300
 - 0s - loss: 0.0575 - acc: 0.2500
Epoch 34/300
 - 0s - loss: 0.0414 - acc: 0.2500
Epoch 35/300
 - 0s - loss: 0.0521 - acc: 0.2500
Epoch 36/300
 - 0s - loss: 0.0534 - acc: 0.2500
Epoch 37/300
 - 0s - loss: 0.0434 - acc: 0.2500
Epoch 38/300
 - 0s - loss: 0.0442 - acc: 0.2500
Epoch 39/300
 - 0s - loss: 0.0444 - acc: 0.2500
Epoch 40/300
 - 0s - loss: 0.0478 - acc: 0.2500
Epoch 41/300
 - 0s - loss: 0.0439 - acc: 0.2500
Epoch 42/300
 - 0s - loss: 0.0344 - acc: 0.2500
Epoch 43/300
 - 0s - loss: 0.0435 - acc: 0.2500
Epoch 44/300
 - 0s - loss: 0.0338 - acc: 0.2500
Epoch 45/300
 - 0s - loss: 0.0517 - acc: 0.2500
Epoch 46/300
 - 0s - loss: 0.0461 - acc: 0.2500
Epoch 47/300
 - 0s - loss: 0.0279 - acc: 0.2500
Epoch 48/300
 - 0s - loss: 0.0656 - acc: 0.2500
Epoch 49/300
 - 0s - loss: 0.0468 - acc: 0.2500
Epoch 50/300
 - 0s - loss: 0.0355 - acc: 0.2500
Epoch 51/300
 - 0s - loss: 0.0523 - acc: 0.2500
Epoch 52/300
 - 0s - loss: 0.0557 - acc: 0.2500
Epoch 53/300
 - 0s - loss: 0.0499 - acc:

Epoch 202/300
 - 0s - loss: 0.0148 - acc: 0.2500
Epoch 203/300
 - 0s - loss: 0.0108 - acc: 0.2500
Epoch 204/300
 - 0s - loss: 0.0145 - acc: 0.2500
Epoch 205/300
 - 0s - loss: 0.0179 - acc: 0.2500
Epoch 206/300
 - 0s - loss: 0.0202 - acc: 0.2500
Epoch 207/300
 - 0s - loss: 0.0235 - acc: 0.2500
Epoch 208/300
 - 0s - loss: 0.0114 - acc: 0.2500
Epoch 209/300
 - 0s - loss: 0.0196 - acc: 0.2500
Epoch 210/300
 - 0s - loss: 0.0095 - acc: 0.2500
Epoch 211/300
 - 0s - loss: 0.0059 - acc: 0.2500
Epoch 212/300
 - 0s - loss: 0.0067 - acc: 0.2500
Epoch 213/300
 - 0s - loss: 0.0100 - acc: 0.2500
Epoch 214/300
 - 0s - loss: 0.0038 - acc: 0.2500
Epoch 215/300
 - 0s - loss: 0.0150 - acc: 0.2500
Epoch 216/300
 - 0s - loss: 0.0171 - acc: 0.2500
Epoch 217/300
 - 0s - loss: 4.0938e-04 - acc: 0.2500
Epoch 218/300
 - 0s - loss: 0.0207 - acc: 0.2500
Epoch 219/300
 - 0s - loss: 0.0074 - acc: 0.2500
Epoch 220/300
 - 0s - loss: 0.0132 - acc: 0.2500
Epoch 221/300
 - 0s - loss: 0.0078 - acc: 0.2500
Epoch 222/300
 -